In [ ]:
!pip install pytorch-lightning~=1.6.5
!pip install --upgrade --force-reinstall altair==4.2.0
!pip install attrs==22.2.0 jsonschema==4.17.3

In [ ]:
# Bước 1: Clone repository
!git clone https://github.com/MinhQuan1909/text-detection-and-recognition.git

In [ ]:
!pip install timm==0.6.13
!pip install nltk==3.7
!pip install lmdb==1.3.0
# !pip install Pillow==9.2.0
!pip install imgaug==0.4.0
!pip install hydra-core==1.2.0
!pip install fvcore==0.1.5.post20221221
!pip install PyYAML==6.0.2
!pip install tqdm==4.64.1
!pip install opencv-python==4.10.0
!pip install streamlit==1.16.0

In [ ]:
%cd text-detection-and-recognition

!pip install gdown --upgrade
!pip install unzip

!gdown --id 1Az9psFV6C1qiqFCL2s6DIBYRKr3Bkjd7 #full
!unzip weights.zip
!rm -rf weights.zip

In [ ]:
!pip install pip==23.0.1

In [ ]:
!pip install streamlit

In [ ]:
from src.parseq.parseq_predict import predict_parseq
from src.parseq.load_model import load_model_parseq

In [ ]:
import torch
device = torch.device("cuda")
parseq, img_transform=load_model_parseq(device=device)

In [ ]:
# from PIL import Image
# import cv2
# image = '/kaggle/input/aic2024-keyframes0304/Keyframes/L03_extra/V001/001750.jpg'
# img = Image.open(image).convert('RGB')

In [ ]:
# (pred, prob) = predict_parseq(parseq=parseq, img_transform=img_transform, image=img, device = 'cuda:0')
# print(f'Model parseq predict this text: "{pred[0]}" with proba {round(prob * 100, 2)}%')

In [ ]:
# !pip install paddlepaddle-gpu==2.5.0 -f https://www.paddlepaddle.org.cn/whl/stable.html
!pip install paddlepaddle-gpu

In [ ]:
!pip install paddleocr

In [ ]:
import os
import cv2
import glob
import json
import torch
# import easyocr
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import shutil

# from vietocr.tool.predictor import Predictor
# from vietocr.tool.config import Cfg

In [ ]:
from paddleocr import PaddleOCR, draw_ocr
import matplotlib.pyplot as plt
import cv2

ocr = PaddleOCR(
    use_pdserving=False,
    use_angle_cls=True,  # Sử dụng phân loại góc để phát hiện góc văn bản
    det=True,            # Bật chế độ phát hiện văn bản
    rec=False,           # Tắt chế độ nhận dạng văn bản
    cls=True,            # Bật phân loại góc
    use_gpu=True,        # Sử dụng GPU nếu có
    lang='vi',           # Ngôn ngữ của văn bản phát hiện là tiếng Việt
    ocr_version="PP-OCRv4",
    show_log=False        # Hiển thị log để theo dõi quá trình
)

In [ ]:
keyframes_dir_list = [f'/kaggle/input/{x}/Keyframes' for x in os.listdir('/kaggle/input') if 'eyframe' in x]
all_keyframe_paths = dict()
for keyframe_dir in keyframes_dir_list:
    for part in sorted(os.listdir(keyframe_dir)):
        data_part = part.split('_')[-2] # L01, L02 for ex
        all_keyframe_paths[data_part] =  dict()

for keyframe_dir in keyframes_dir_list:
    for data_part in sorted(all_keyframe_paths.keys()):
        data_part_path = f'{keyframe_dir}/{data_part}_extra'
        if os.path.isdir(data_part_path):
            video_dirs = sorted(os.listdir(data_part_path))
            video_ids = [video_dir.split('_')[-1] for video_dir in video_dirs]
            for video_id, video_dir in zip(video_ids, video_dirs):
                keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.jpg'))
                all_keyframe_paths[data_part][video_id] = keyframe_paths

## inference

In [ ]:
%cd ..

In [ ]:
bs = 16
save_dir = './ocr'
crop_dir = './crop'  # Thư mục lưu ảnh đã cắt

# Tạo các thư mục nếu chưa có
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

if not os.path.exists(crop_dir):
    os.mkdir(crop_dir)

keys = sorted(all_keyframe_paths.keys())

for key in tqdm(keys):
    video_keyframe_paths = all_keyframe_paths[key]
    video_ids = sorted(video_keyframe_paths.keys())

    if not os.path.exists(os.path.join(save_dir, key)):
        os.mkdir(os.path.join(save_dir, key))

    for video_id in tqdm(video_ids):
        video_keyframe_path = video_keyframe_paths[video_id]
        video_ocr_results = []

        for i in range(0, len(video_keyframe_path), bs):
            image_paths = video_keyframe_path[i:i+bs]

            # Thực hiện OCR trên từng ảnh
            for img_path in image_paths:
                # Đọc ảnh gốc
                img = cv2.imread(img_path)
                np_img = np.array(img)

                # Cắt bỏ phần nền đỏ chữ trắng
                height, width, _ = np_img.shape
                crop_height = int(0.85 * height)
                image_cropped = np_img[:crop_height, :]
                
                # Lưu ảnh đã cắt vào thư mục crop/
                name = os.path.basename(img_path)
                cropped_img_path = f"{crop_dir}/{name}"
                cv2.imwrite(cropped_img_path, image_cropped)
#                 print(f"Saved cropped image to {cropped_img_path}")

                # Phát hiện văn bản và góc bằng PaddleOCR trên ảnh đã cắt
                result = ocr.ocr(cropped_img_path, cls=True)

                if result is not None and len(result) > 0 and result[0] is not None:
                    refined_result = []
                    for line in result:
                        for res in line:
                            if len(res) == 3:
                                box, (text, score), angle = res  # Khi có angle
#                                 print(f"Detected angle: {angle}")  # In ra góc phát hiện
                            else:
                                box, (text, score) = res  # Khi không có angle
                                angle = 0

                            # `box` là danh sách 4 điểm (x, y) đại diện cho bounding box của đoạn văn bản
                            box = [tuple(map(int, point)) for point in box]
                            #box = 

                            # Cắt vùng ảnh theo bounding box để nhận diện bằng VietOCR
                            x_min = min(box[0][0], box[1][0])
                            x_max = max(box[2][0], box[3][0])
                            y_min = min(box[0][1], box[3][1])
                            y_max = max(box[1][1], box[2][1])
                            roi = image_cropped[y_min:y_max, x_min:x_max]

                            # Kiểm tra nếu roi không bị trống
                            if roi.size > 0:
                                # Nếu phát hiện góc và góc khác 0, xoay ROI theo góc phát hiện
                                if angle != 0:
                                    center = (roi.shape[1] // 2, roi.shape[0] // 2)
                                    M = cv2.getRotationMatrix2D(center, angle, 1.0)
                                    roi = cv2.warpAffine(roi, M, (roi.shape[1], roi.shape[0]))
            
                                # Chuyển đổi ROI sang ảnh PIL để sử dụng với parseq
                                roi_pil = Image.fromarray(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))
#                                 recognized_text = detector.predict(roi_pil) 
                                (pred, prob) = predict_parseq(parseq=parseq, img_transform=img_transform, image=roi_pil, device = 'cuda:0')
                                recognized_text = pred[0]
                                refined_result.append(recognized_text)

                    video_ocr_results.append(refined_result)

        # Lưu kết quả OCR vào file JSON
        with open(f'{save_dir}/{key}/{video_id}.json', "w", encoding='utf-8') as jsonfile:
            json.dump(video_ocr_results, jsonfile, ensure_ascii=False)

In [ ]:
!rm -rf /kaggle/working/crop
!rm -rf /kaggle/working/text-detection-and-recognition